## Homework Questions

In [1]:
#1. Find how many cells took longer to find instanteous threshold than they did to find rheobase (ie more SSFINEST than LSFINEST sweeps).
#2. Show the distribution of hemispheres all the cells patched in 2017 with the age and sex of the mice used.
#3. Look for cells that do not have a qc reviewer and failed qc for project T301. What are the cell level qc failures? (access, gohm seal, blowout, input resistance)
#4. Find all the H301 cells where DiJon was stage 1 reviewer that manually passed qc and the cells where Lisa was stage 2 review that manually failed qc.

In [2]:
import pg8000          #pg8000 access SQL databases
import pandas as pd    #pandas will be needed to work in a dataframe

In [3]:
#Code from Agata
#Opens LIMS, makes a query and then closes LIMS after

def _connect(user="limsreader", host="limsdb2", database="lims2", password="limsro", port=5432):
    conn = pg8000.connect(user=user, host=host, database=database, password=password, port=port)
    return conn, conn.cursor()

def _select(cursor, query):
    cursor.execute(query)
    columns = [ d[0] for d in cursor.description ]
    return [ dict(zip(columns, c)) for c in cursor.fetchall() ]

def limsquery(query, user="limsreader", host="limsdb2", database="lims2", password="limsro", port=5432):
    """A function that takes a string containing a SQL query, connects to the LIMS database and outputs the result."""
    conn, cursor = _connect(user, host, database, password, port)
    try:
        results = _select(cursor, query)
    finally:
        #THESE ARE IMPORTANT!!!!!! (Every query needs to be closed when done)
        #This closes query
        cursor.close()             
        conn.close()
    return results


#This last function will take our query results and put them in a dataframe so that they are easy to work with
def get_lims_dataframe(query):
    '''Return a dataframe with lims query'''
    result = limsquery(query)
    try:
        data_df = pd.DataFrame(data=result, columns=result[0].keys())
    except IndexError:
        print "Could not find results for your query."
        data_df = pd.DataFrame()
    return data_df

### Question 1: Find how many cells took longer to find instanteous threshold than they did to find rheobase (ie more SSFINEST than LSFINEST sweeps).

In [7]:
my_query = "SELECT * FROM ephys_sweeps LIMIT 1"
#asterisk(*) = all columns
#specimens = table name
#LIMIT 1 = limits search to first row

my_result = limsquery(my_query)
first_element = my_result[0]
#first_element is created to only look at the first row

print first_element.keys()
#keys = column names
#Shows all columns names 

['stimulus_interval', 'updated_at', 'num_spikes', 'id', 'pre_vm_mv', 'stimulus_duration', 'stimulus_start_time', 'slow_noise_rms_mv', 'stimulus_amplitude', 'peak_deflection', 'bridge_balance_mohm', 'stimulus_units', 'workflow_state', 'specimen_id', 'sweep_number', 'vm_delta_mv', 'leak_pa', 'pre_noise_rms_mv', 'ephys_stimulus_id', 'post_noise_rms_mv', 'post_vm_mv', 'created_at', 'slow_vm_mv']


In [5]:
my_query = "SELECT * FROM specimens cell LIMIT 1"
#asterisk(*) = all columns
#specimens = table name
#LIMIT 1 = limits search to first row

my_result = limsquery(my_query)
first_element = my_result[0]
#first_element is created to only look at the first row

print first_element.keys()
#keys = column names
#Shows all columns names 

['cell_depth', 'ephys_roi_result_id', 'parent_y_coord', 'reference_space_id', 'updated_at', 'cell_label', 'preparation_method_id', 'parent_x_coord', 'location_id', 'id', 'cortex_layer_id', 'plane_of_section_id', 'frozen_at', 'flipped_specimen_id', 'data', 'pinned_radius', 'rna_integrity_number', 'histology_well_name', 'created_by', 'priority', 'parent_id', 'ephys_start_time_sec', 'project_id', 'alignment3d_id', 'carousel_well_name', 'patched_cell_container', 'updated_by', 'cell_prep_id', 'biophysical_model_state', 'barcode', 'storage_directory', 'x_coord', 'tissue_ph', 'specimen_preparation_method_id', 'donor_id', 'operation_id', 'ephys_neural_tissue_plan_id', 'structure_id', 'parent_z_coord', 'facs_well_id', 'name', 'y_coord', 'normalization_group_id', 'postmortem_interval_id', 'specimen_set_id', 'created_at', 'tissue_processing_id', 'ephys_cell_plan_id', 'hemisphere_id', 'ephys_qc_result', 'cell_reporter_id', 'task_flow_id', 'external_specimen_name']


In [6]:
my_query = "SELECT * FROM ephys_stimuli LIMIT 1"
#asterisk(*) = all columns
#specimens = table name
#LIMIT 1 = limits search to first row

my_result = limsquery(my_query)
first_element = my_result[0]
#first_element is created to only look at the first row

print first_element.keys()
#keys = column names
#Shows all columns names 

['description', 'created_at', 'ephys_stimulus_type_id', 'updated_at', 'amplitude', 'id']


### Question 2: Show the distribution of hemispheres all the cells patched in 2017 with the age and sex of the mice used.

### Question 3: Look for cells that do not have a qc reviewer and failed qc for project T301. What are the cell level qc failures? (access, gohm seal, blowout, input resistance)

##### Table with QC reviewers & QC failures

In [9]:
my_query = "SELECT * FROM ephys_roi_results LIMIT 1"
#asterisk(*) = all columns
#specimens = table name
#LIMIT 1 = limits search to first row

my_result = limsquery(my_query)
first_element = my_result[0]
#first_element is created to only look at the first row

print first_element.keys()
#keys = column names
#Shows all columns names 

['rig_name', 'ephys_qc_criteria_id', 'failed_bad_rs', 'updated_at', 'storage_directory', 'electrode_0_pa', 'input_resistance_mohm', 'id', 'stage2_reviewer_id', 'blowout_mv', 'failed_other', 'sampling_rate', 'input_access_resistance_ratio', 'failed_no_seal', 'workflow_state', 'ephys_specimen_roi_plan_id', 'initial_access_resistance_mohm', 'qc_notes', 'recording_date', 'created_at', 'seal_gohm', 'published_at', 'failed_clogged_pipette', 'stage1_reviewer_id', 'failed_electrode_0', 'notes']


In [15]:
my_query = "SELECT stage1_reviewer_id, stage2_reviewer_id, initial_access_resistance_mohm, seal_gohm, \
            blowout_mv, input_resistance_mohm FROM ephys_roi_results LIMIT 1"
#asterisk(*) = all columns
#specimens = table name
#LIMIT 1 = limits search to first row

my_result = limsquery(my_query)
first_element = my_result[0]
#first_element is created to only look at the first row

print first_element
#Shows first row of entry 

{'stage2_reviewer_id': 485591509, 'seal_gohm': 1.6066233600000002, 'blowout_mv': 3.3822432160377502, 'stage1_reviewer_id': 525757437, 'input_resistance_mohm': 739.51104, 'initial_access_resistance_mohm': 17.023903999999998}


In [22]:
#Cell level qc failures = (initial_access_resistance_mohm, seal_gohm, blowout_mv, input_resistance_mohm)
#Need Stage 1 reviewer, stage 2 reviewer
#Need project code, specimen name
#You need project code from a different table and join these tables to together

##### Table with project codes

In [12]:
my_query = "SELECT * FROM projects LIMIT 1"
#asterisk(*) = all columns
#specimens = table name
#LIMIT 1 = limits search to first row

my_result = limsquery(my_query)
first_element = my_result[0]
#first_element is created to only look at the first row

print first_element.keys()
#keys = column names
#Shows all columns names 

['code', 'file_storage', 'name', 'current_directory', 'non_cached_schema_name', 'subdirectory_count', 'created_at', 'updated_at', 'process_triggers', 'trigger_dir', 'incoming_directory', 'failed_trigger_dir', 'id']


In [21]:
my_query = "SELECT code FROM projects LIMIT 1"
#asterisk(*) = all columns
#specimens = table name
#LIMIT 1 = limits search to first row

my_result = limsquery(my_query)
first_element = my_result[0]
#first_element is created to only look at the first row

print first_element
#Shows first row of entry 

{'code': u'0200'}


### Question 4: Find all the H301 cells where DiJon was stage 1 reviewer that manually passed qc and the cells where Lisa was stage 2 review that manually failed qc.

In [16]:
#Use same table from question 3 with different filters

In [8]:
my_query = "SELECT * FROM ephys_roi_results LIMIT 1"
#asterisk(*) = all columns
#specimens = table name
#LIMIT 1 = limits search to first row

my_result = limsquery(my_query)
first_element = my_result[0]
#first_element is created to only look at the first row

print first_element.keys()
#keys = column names
#Shows all columns names 

['rig_name', 'ephys_qc_criteria_id', 'failed_bad_rs', 'updated_at', 'storage_directory', 'electrode_0_pa', 'input_resistance_mohm', 'id', 'stage2_reviewer_id', 'blowout_mv', 'failed_other', 'sampling_rate', 'input_access_resistance_ratio', 'failed_no_seal', 'workflow_state', 'ephys_specimen_roi_plan_id', 'initial_access_resistance_mohm', 'qc_notes', 'recording_date', 'created_at', 'seal_gohm', 'published_at', 'failed_clogged_pipette', 'stage1_reviewer_id', 'failed_electrode_0', 'notes']
